[컨볼루션 신경망 레이어 이야기](https://tykimos.github.io/2017/01/27/CNN_Layer_Talk/)  
하기 내용은 상기 링크를 정리한것.

# 컨볼루션 신경망 레이어 이야기

## 필터로 특징을 뽑아주는 컨볼루션(Convolution) 레이어
케라스에서 제공되는 컨볼루션 레이어 종류에도 여러가지가 있으나 영상 처리에 주로 사용되는 Conv2D 레이어를 살펴본다.  
레이어는 영상인식에 주로 사용되며, 필터가 탑재된다.  
아래는 그에 따른 Conv2D클래스 사용예제 이다.

```
Conv2D(32, (5, 5), padding='valid', input_shape=(28, 28, 1), activtion='relu')
```